# Evaluación de modelos de texto generativo

Se comenzará la sección configurando el LLM para la generación de texto a partir del código implementado en la sección 3.

![Texto alternativo](./imgs/5.2.png)

La  siguiente  subsección  resume  la  generación  de  texto  que se configuró al final de la sección 3, antes  de  sumergirnos  en  la  evaluación  del  texto  y  el  cálculo  
de  las  pérdidas  de  entrenamiento  y  validación  en  las  subsecciones  posteriores.

### 1.1.1 Uso de GPT para generar texto

En esta subsección se configura el LLm y se recapitula brevemente el proceso de generación de texto

In [3]:
import sys
import os

# Obtiene la ruta de la carpeta principal del proyecto (subiendo un nivel desde seccion05)
ruta_proyecto_principal = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Añade esta ruta a la lista de lugares donde Python busca módulos
if ruta_proyecto_principal not in sys.path:
    sys.path.append(ruta_proyecto_principal)



In [4]:
import torch
from seccion04_ImplementacionGPTGeneracionTexto.gptModel import GPTModel
from seccion04_ImplementacionGPTGeneracionTexto.gptConfig124M import GPT_CONFIG_124M

#importacion de librerias necesarias y acortamiento del context length de la configuracion de GPT (1024 tokens a  256)
GPT_CONFIG_124M["context_length"] = 256
print(GPT_CONFIG_124M)

{'vocab_size': 50257, 'context_length': 256, 'emb_dim': 768, 'n_heads': 12, 'n_layers': 12, 'drop_rate': 0.1, 'qkv_bias': False}


In [5]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

Esta  modificación  reduce  las  demandas  computacionales  del  entrenamiento  del  modelo,  haciendo  posible  realizar  el  entrenamiento  en  una  computadora  portátil  estándar.

Originalmente,  el  modelo  GPT2,  con  124  millones  de  parámetros,  se  configuró  para  gestionar  hasta  1024  tokens. 

Utilizando  la  instancia  GPTmodel ,  adoptamos  la  función  generate_text_simple  presentada anteriorment e se introducen dos  funciones  útiles:  text_to_token_ids  y  token_ids_to_text.  Estas  funciones  facilitan  la  conversión  entre  representaciones  de  texto  y  tokens.

![Texto alternativo](./imgs/5.3.png)

- Primero,  el  tokenizador  convierte  el  texto  de  entrada  en  una  serie  de  identificadores  de  token.
- Segundo,  el  modelo  recibe  estos  identificadores  de  token  y  genera  los  logits  correspondientes,  que  son  vectores  que  representan  la  distribución  de  probabilidad  de  cada  token  del  vocabulario.
-  Tercero,  estos  logits  se  convierten  de  nuevo  en  identificadores  de  token,  que  el  tokenizador  decodifica  en  texto  legible,  completando  así  el  ciclo  de  entrada  a  salida  textual.

In [6]:
from seccion04_ImplementacionGPTGeneracionTexto.generateTextSimple import generate_text_simple
import tiktoken

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    # 1. squeeze(0): Quita la dimensión del "lote" que añadió unsqueeze.
    #Es como sacar la lista de la caja para poder leerla.
    #Ej: tensor([[25134]]) -> tensor([25134]), forma: [1]
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) 
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    # 2. Convierte el tensor 1D a una lista y luego a texto.
    # Ej: tensor([25134]) -> [25134] -> "hola"

    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))



Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


El modelo sigue sin producir texto coherente porque todavia no se ha entrenado.

Para  definir  qué  hace  que  un  texto  sea  "coherente"  o  de  "alta  calidad", se debe implementar un  método  numérico  para  evaluar  el  contenido  generado.  Este  enfoque  nos  permitirá  monitorear  y  mejorar  el  rendimiento  del  modelo  durante  su  proceso  de  entrenamiento.

### 1.1.2 Cálculo de la perdida de generación de texto

En esta subsección se exploraran técnicas para evaluar numericamnte la calidad del texto generado durante el entrenamiento mediante el cálculo de la llamada función de pérdidda de generación de texto.

![Texto alternativo](./imgs/5.1.png)

Es necesario realizar todos estos pasos indicados en la figura antes de calcular una pérdida que mida la calidad del texto generado.

La figura describe  el  proceso  de  generación  de  texto  con  un  vocabulario  reducido  de  7  tokens  para  que  esta  imagen  quepa  en  una  sola  página.  Sin  embargo,  GPTModel  trabaja  con  un  vocabulario  mucho  mayor,  compuesto  por  50 257  palabras;  por  lo  tanto,  los  ID  de  token  en  los  siguientes  códigos  estarán  comprendidos  entre  0  y  50 256,  en  lugar  de  entre  0  y  6.

Definición de dos entradas con sus objetivos:

In [7]:
#Paso 1
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                        [40, 1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [107,  588, 11311]]) #  " really like chocolate"]

Los  objetivos  son  las  entradas,  pero  desplazados  una  posición  hacia  adelante.
Esta  estrategia  de  desplazamiento  es  crucial  para  enseñar  al  modelo  a  predecir  el  siguiente  token  en  una  secuencia. Cuando  introducimos  las  entradas  en  el  modelo  para  calcular  vectores  logit  para  los  dos  ejemplos  de  entrada,  cada  uno  compuesto  por  tres  tokens,  y  aplicamos  la  función  softmax  para  transformar  estos  valores  logit  en  puntuaciones  de  probabilidad. 

In [8]:
#Paso 2: logits a scores de probabilidad
with torch.no_grad():
    logits = model(inputs)
probas = torch.softmax(logits, dim=-1)
print(probas.shape) 

#Paso 3 y 4: argmax para obtener el correspondiente token ID 
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Ids token: ", token_ids)

torch.Size([2, 3, 50257])
Ids token:  tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


Dado  que  se tiene  2  lotes  de  entrada,  cada  uno  con  3  tokens,  aplicar  la  función  argmax  a  los  puntajes  de  probabilidad  produce  2  conjuntos  de  salidas,  cada  uno  con  3  identificaciones  de  tokens  predichas.

In [9]:
#Paso 5: Convertir IDs a texto
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}") #verdadera salida
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}") #salida del modelo

Targets batch 1:  effort moves you
Outputs batch 1:  Armed heNetflix


El  modelo  produce  texto  aleatorio  diferente  del  texto  objetivo  porque  aún  no  se  ha  entrenado. 

Ahora se debe evaluar numéricamente el rendimiento del texto generado por el modelo mediante la llamada pérdida. Esto no es solo útil para medir la calidad del texto generado, sino que también es un elemento fundamental para implementar posteriormente la función de entrenamiento, que se utiliza para actualizar el peso del modelo y mejorar el texto generado.

![Texto alternativo](./imgs/5.5.png)

Parte  del  proceso  de  evaluación  de  texto consiste  en  medir  la  distancia  entre  los  tokens  generados  y  las  predicciones  correctas  (objetivos). La función de entrenamiento que se implementará utilizará esta información para ajustar los pesos del modelo y generar un texto más similar.
El  entrenamiento  del  modelo  busca  aumentar  la  probabilidad  softmax  en  las  posiciones  de  índice  correspondientes  a  los  ID  de  token  objetivo  correctos.

![Texto alternativo](./imgs/5.6.png)

Antes  del  entrenamiento,  el  modelo  genera  vectores  aleatorios  de  probabilidad  del  siguiente  token.  El  objetivo  del  entrenamiento  del  modelo  es  garantizar  que  los  valores  de  probabilidad  correspondientes  a  los  ID  de  token  objetivo  resaltados  sean maximizado.

Para  cada  uno  de  los  dos  textos  de  entrada,  podemos  imprimir  las  puntuaciones  de  probabilidad  softmax  iniciales correspondiente  a  los  tokens  de  destino  a  través  del  siguiente  código:

In [10]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]] #porbabilidades asociadas a los objetivos (cuales inidice son los que se quieren que salgan)
print("Text 1:", target_probas_1)
text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

Text 1: tensor([7.4541e-05, 3.1061e-05, 1.1563e-05])
Text 2: tensor([3.9108e-05, 5.6776e-05, 4.7559e-06])


El  objetivo  de  entrenar  un  LLM  es  maximizar  estos  valores,  intentando  acercarlos  lo  más  posible  a  una  probabilidad  de  1.  De  esta  manera,  garantizamos  que  el  LLM  elija  consistentemente  el  token  objetivo  (esencialmente,  la  siguiente  palabra  de  la  oración)  como  el  siguiente  token  que  genera.

***Retropropagación***
¿Cómo  maximizamos  los  valores  de  probabilidad  softmax  correspondientes  a  los  tokens  objetivo?  
En  resumen,  actualizamos  los  pesos  del  modelo  para  que  este  genere  valores  más  altos  para  los  respectivos  ID  de  token  que  queremos  generar.  La  actualización  de  los  pesos  se  realiza  mediante  un  proceso  llamado  retropropagación,  una  técnica  estándar  para  el  entrenamiento  de  redes  neuronales  profundas.

La  retropropagación  requiere  una  función  de  pérdida  que  calcula  la  diferencia  entre  el  resultado  previsto  del  modelo  (en  este  caso,  las  probabilidades  correspondientes  a  los  ID  de  token  objetivo)  y  el  resultado  deseado  real.  Esta  función  de  pérdida  mide  la  distancia  entre  las  predicciones  del  modelo  y  los  valores  objetivo.


***Perdida de entropia cruzada***

![Texto alternativo](./imgs/5.7.png)

Los  pasos  1  a  3  calculan  las  probabilidades  de  los  tokens  correspondientes  a  los  tensores  objetivo.  Estas  probabilidades  se  transforman  mediante  un  logaritmo  y  se  promedian  en  los  pasos  4  a  6.

In [11]:
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([ -9.5042, -10.3796, -11.3677, -10.1492,  -9.7764, -12.2561])


Trabajar  con  logaritmos  de  puntuaciones  de  probabilidad  es  más  manejable  en  la  optimización  matemática  que  manejar  las  puntuaciones  directamente. 
A  continuación,  combinamos  estas  probabilidades  logarítmicas  en  una  única  puntuación  calculando  el  promedio

In [12]:
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

tensor(-10.5722)


El  objetivo  es  obtener  la  probabilidad  logarítmica  promedio  lo  más  cerca  posible  de  0  actualizando  los  pesos  del  modelo  como  parte  del  proceso  de  entrenamiento.

Sin  embargo,  en  el  aprendizaje  profundo,  la  práctica  común  no  es  aumentar  la  probabilidad  logarítmica  promedio  hasta  0,  sino  reducir  la  probabilidad  logarítmica  promedio  negativa  a  0.  La  probabilidad  logarítmica  promedio  negativa  es  simplemente  la  probabilidad  logarítmica  promedio  multiplicada  por  1,  lo  que  corresponde  al  paso  6. 

In [13]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(10.5722)


El  término  para  este  valor  negativo,  10,7940  convirtiéndose  en  10,7940,  se  conoce  como  la  cruz depérdida  de  entropía  en  el  aprendizaje  profundo.

***PyTorch  resulta  útil  en  este  caso,  ya  que  tiene  una  función  cross_entropy  incorporada  que  se  encarga  de  todos  estos  6  pasos***

### Pérdida de entropia cruzada
En  esencia,  la  pérdida  de  entropía  cruzada  es  una  medida  popular  en  el  aprendizaje  automático  y  el  aprendizaje  profundo  que  mide  la  diferencia  entre  dos  distribuciones  de  probabilidad:  típicamente,  la  distribución  real  de  etiquetas  (aquí,  tokens  en  un  conjunto  de  datos)  y  la  distribución  prevista  a  partir  de  un  modelo  (por  ejemplo,  las  probabilidades  de  token  generadas  por  un  LLM).
En  el  contexto  del  aprendizaje  automático  y  específicamente  en  marcos  como  PyTorch,  la  función  cross_entropy  calcula  esta  medida  para  resultados  discretos,  que  es  similar  a  la  probabilidad  logarítmica  promedio  negativa  de  los  tokens  de  destino  dadas  las  probabilidades  de  token  generadas  por  el  modelo,  lo  que  hace  que  los  términos  entropía  cruzada  y  probabilidad  logarítmica  promedio  negativa  estén  relacionados  y  a  menudo  se  usen  indistintamente  en  la  práctica.

Anteriormente,  se aplicó la  función  softmax,  seleccionamos  los  puntajes  de  probabilidad  correspondientes  a  los  identificadores  de  destino  y  calculamos  las  probabilidades  logarítmicas  promedio  negativas.
La  función  cross_entropy  de  PyTorch  se  encargará  de  todos  estos  pasos  por  nosotros:

In [14]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()
print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])
tensor(10.5722)


La pérdida resultante es la misam que se obtuvo al aplicar manualmente los pasos individuales.

### 1.1.3 Cálculo de las pérdidas del conjunto de entrenamieto y validación

En esta subsección , primero se prepararan los cconjuntos de datos de entrenamiento y validación que se utilizará para entrenar el LLM.
Seguidamente se calculará la entropía cruzada para los conjuntos.

![Texto alternativo](./imgs/5.8.png)


Para poder calcular la pérdida en los conjuntos de entrenamiento y validación, se va a utilizar un conjunto de datos pequeño del cuento **The Verdict**.

### EL costo de la formación previa de LLMS
Para  poner  en  perspectiva  la  escala  de  nuestro  proyecto,  consideremos  el  entrenamiento  del  modelo  Llama  2  de  7  mil  millones  de  parámetros,  un  modelo  LLM  relativamente  popular  y  de  libre  acceso.  Este  modelo  requirió  184 320  horas  de  GPU  en  costosas  GPU  A100,  procesando  2  billones  de  tokens.  Al  momento  de  escribir  este  artículo,  ejecutar  un  servidor  en  la  nube  8xA100  en  AWS  cuesta  alrededor  de  30$   por  hora.  Una  estimación  aproximada  sitúa  el  costo  total  de  entrenamiento  de  dicho  modelo  LLM  en  unos  690 000$  (calculado  como  184 320  horas  divididas  entre  8  y  multiplicadas  por  30$).

In [15]:
file_path = "../txt/The_Verdict.txt"
with open(file_path, "r", encoding="utf-8") as f:
    text_data = f.read()

#Despues de cargar el dataset, se puede chequear el número de caracteres y tokens
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))
print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 20483
Tokens: 5147


Con  tan  solo  5145  tokens,  el  texto  podría  parecer  demasiado  pequeño  para  entrenar  un  LLM,  pero  como  se  mencionó  anteriormente,  su  propósito  es  educativo  para  que  podamos  ejecutar  el  código  en  minutos  en  lugar  de  semanas. 

A  continuación,  se dividen  el  conjunto  de  datos  en  un  conjunto  de  entrenamiento  y  uno  de  validación,  y utilizando los  cargadores  de  datos  de la sección 2  para  preparar  los  lotes  para  el  entrenamiento  LLM. 

![Texto alternativo](./imgs/5.9.png)

En los cargadores de datos que se van a crear se va a establecer max_length igual a la longitud de contexto de 256 tokens que admite el LLM para que el LLm vea textos más largos durante el entrenemiento.

Para  implementar  la  división  y  carga  de  datos  visualizada  en  la  Figura,  primero  definimos  un  train_ratio  para  utilizar  el  90  %  de  los  datos  para  entrenamiento  y  el  10  %  restante  como  datos  de  validación  para  la  evaluación  del  modelo  durante  el  entrenamiento:

In [16]:
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[: split_idx]
val_data = text_data[split_idx :]

Usando train data y val data, se pueden crear ahora los respectivos data loader reutilizando las funciones de la sección 2.

In [17]:
from seccion02_TrabajarDatosTexto.dataloader_v1 import create_dataloader_v1
torch.manual_seed(123)
train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)
val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [18]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)
    
print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

len(train_loader)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


9

Según  el  código  de  salida  anterior,  tenemos  9  lotes  de  conjuntos  de  entrenamiento  con  2  muestras  y  256  tokens  cada  uno.  Dado  que  solo  asignamos  el  10  %  de  los  datos  a  la  validación,  solo  hay  un  lote  de  validación  con  2  ejemplos  de  entrada.

Como  se  esperaba,  los  datos  de  entrada  (x)  y  los  datos  de  destino  (y)  tienen  la  misma  forma  (el  tamaño  del  lote  multiplicado  por  la  cantidad  de  tokens  en  cada  lote)  ya  que  los  destinos  son  las  entradas  desplazadas  una  posición.

Implementación de una  función  de  utilidad  para  calcular  la  pérdida  de  entropía  cruzada  de  un  lote  determinado  devuelto  a  través  del  cargador  de  entrenamiento  y  validación:

In [19]:
def calc_loss_batch(input_batch, taget_batch, model, device):
    input_batch, target_batch = input_batch.to(device), taget_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss

Ahora se puede usar esta  función  de  utilidad  calc_loss_batch ,  que  calcula  la  pérdida  de  un  solo  lote,  para  implementar  la  siguiente  función  calc_loss_loader  que  calcula  la  pérdida  de  todos  los  lotes  muestreados  por  un  cargador  de  datos  determinado.

In [ ]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)                 #Iterativo sobre el numero de lotes si no se especifica un numero fijo de lotes           
    else:
        num_batches = min(num_batches, len(data_loader))          #reducir el numero de lotes para que coicida con el numero total de lotes en el cargador de datos

    for i, (input_batch, target_batch) in enumerate(data_loader):   
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()                           #suma de perdidas por cada lote
        else:
            break
    return total_loss / num_batches     #promedio de la perdida en todos los lotes

De  forma  predeterminada,  la  función  calc_loss_batch  itera  sobre  todos  los  lotes  de  un  cargador  de  datos  determinado,  acumula  la  pérdida  en  la  variable  total_loss  y,  a  continuación,  calcula  y  promedia  la  pérdida  sobre  el  número  total  de  lotes.  Como  alternativa, se puede especificar  un  número  menor  de  lotes  mediante  num_batches  para  agilizar  la  evaluación  durante  el  entrenamiento  del  modelo.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
with torch.no_grad(): #deshabilitar el seguimiento de gradiente ya que no se esta entrenando
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 10.987583796183268
Validation loss: 11.025263786315918


Los  valores  de  pérdida  son  relativamente  altos  porque  el  modelo  aún  no  se  ha  entrenado.  A  modo  de  comparación,  la  pérdida  se  aproxima  a  cero  si  el  modelo  aprende  a  generar  los  siguientes  tokens  a  medida  que  aparecen  en  los  conjuntos  de  entrenamiento  y  validación.

Ahora  que  se tiene  una  forma  de  medir  la  calidad  del  texto  generado,  en  la  siguiente  sección  se entrenará  al  LLM  para  reducir  esta  pérdida  para  que  sea  mejor  en  la  generación  de  texto.

![Texto alternativo](./imgs/5.10.png)

[Entrenamiento de un LLM](./2_entrenando_llm.ipynb)